STGCN trained on simulated streamflow.

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 
from datetime import datetime, timedelta
import netCDF4 as nc
import torch
from torch import nn, utils
from torch.utils.tensorboard import SummaryWriter
from src import load_data, evaluate, conv_lstm, datasets, utils, stgcn
import random
import pickle
import json
import networkx as nx

time_stamp = datetime.now().strftime('%Y%m%d-%H%M%S')
time_stamp

'20190825-153140'

In [2]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='../log.out', mode='a')
chandler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('%(asctime)s - {} - %(message)s'.format(time_stamp))
fhandler.setFormatter(formatter)
chandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.addHandler(chandler)
logger.setLevel(logging.INFO)

In [3]:
USE_CUDA = False
if torch.cuda.is_available():
    print('CUDA Available')
    USE_CUDA = True
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
device = torch.device('cuda:0' if USE_CUDA else 'cpu')
num_devices = torch.cuda.device_count() if USE_CUDA else 0
logger.warning('cuda devices: {}'.format(list(torch.cuda.get_device_name(i) for i in range(num_devices))))
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

CUDA Available
2019-08-25 15:31:40,335 - 20190825-153140 - cuda devices: ['Tesla V100-SXM2-16GB']


In [4]:
partitioning_strategy = 'distance'  # 'distance' or 'unilabel', see https://arxiv.org/abs/1801.07455
max_hops = 3 if partitioning_strategy == 'distance' else None
rdrs_vars = list(range(8))
agg = ['minmax','minmax','minmax','minmax','sum','minmax','minmax','minmax']
include_month = True
dem, landcover, soil, groundwater = False, False, False, False
landcover_types = []
seq_len = 14
seq_steps = 1

train_start = datetime.strptime('2010-01-01', '%Y-%m-%d') + timedelta(days=seq_len * seq_steps)  # first day for which to make a prediction in train set
train_end = '2012-12-31'
test_start = '2013-01-01'
test_end = '2014-12-31'
spatial_val = True  # Whether the val set does spatial or temporal validation
val_fraction = 0.1 if not spatial_val else None

In [5]:
with open('../data/simulations_shervan/subbasins.geojson', 'r') as f:
     subbasin_shapes = json.loads(f.read())

subbasin_graph = utils.create_subbasin_graph()
component_graph = subbasin_graph.copy()
component_graph.remove_nodes_from(['sub-1', 'sub1', 'sub474'])  # remove Lake Erie and sink to get connected components
connected_components = sorted(sorted(c) for c in nx.connected_components(nx.Graph(component_graph)))

# Split into train/test/val regions
test_subbasins = [1, 474]
train_subbasins = []
val_subbasins = []
train_components = []
for component in connected_components:
    max_x = -999
    for node in component:
        subbasin = list(s['properties'] for s in subbasin_shapes['features'] if 'sub' + str(s['properties']['SubId']) == node)[0]
        max_x = max(max_x, subbasin['INSIDE_X'])
    if max_x < -81.9:
        train_components.append(list(int(c[3:]) for c in component))
        train_subbasins += train_components[-1]
    elif -80.6 > max_x and max_x >= -81.9:
        val_subbasins += list(int(c[3:]) for c in component)
    else:
        test_subbasins += list(int(c[3:]) for c in component)
        
if not spatial_val:  # if no spatial validation, use same graph but different samples
    train_subbasins += val_subbasins
    val_subbasins = train_subbasins
train_subbasins, val_subbasins, test_subbasins = sorted(train_subbasins), sorted(val_subbasins), sorted(test_subbasins)

# For each train-component, get its nodes' indices into train_subbasins. 
# We'll use them to create the adjacency matrix of the component during training.
train_component_indices = []
for component in train_components:
    train_component_indices.append(sorted(train_subbasins.index(c) for c in component))

train_subgraph = subbasin_graph.subgraph(list('sub' + str(t) for t in train_subbasins))
val_subgraph = subbasin_graph.subgraph(list('sub' + str(t) for t in val_subbasins))
test_subgraph = subbasin_graph.subgraph(list('sub' + str(t) for t in test_subbasins))
if partitioning_strategy == 'unilabel':
    train_adjacency = torch.unsqueeze(torch.from_numpy(nx.to_numpy_array(train_subgraph, nodelist=list('sub' + str(t) for t in train_subbasins))), 0).float().to(device)
    val_adjacency = torch.unsqueeze(torch.from_numpy(nx.to_numpy_array(val_subgraph, nodelist=list('sub' + str(t) for t in val_subbasins))), 0).float().to(device)
    test_adjacency = torch.unsqueeze(torch.from_numpy(nx.to_numpy_array(test_subraph, nodelist=list('sub' + str(t) for t in test_subbasins))), 0).float().to(device)
elif partitioning_strategy == 'distance':  # use distances in upstream-graph, i.e. in reversed downstream-graph
    train_adjacency = utils.create_hop_matrix(train_subgraph.reverse(), max_hops, list('sub' + str(t) for t in train_subbasins)).float().to(device)
    val_adjacency = utils.create_hop_matrix(val_subgraph.reverse(), max_hops, list('sub' + str(t) for t in val_subbasins)).float().to(device)
    test_adjacency = utils.create_hop_matrix(test_subgraph.reverse(), max_hops, list('sub' + str(t) for t in test_subbasins)).float().to(device)
else:
    raise Exception('Unsupported partitioning strategy')

subbasins = sorted(set(train_subbasins + test_subbasins + val_subbasins))

In [6]:
train_dataset = datasets.SubbasinAggregatedDataset(rdrs_vars, train_subbasins, seq_len, seq_steps, train_start, train_end, aggregate_daily=agg, include_months=include_month, 
                                                   dem=dem, landcover=landcover, soil=soil, groundwater=groundwater, landcover_types=landcover_types)
if spatial_val:
    val_dataset = datasets.SubbasinAggregatedDataset(rdrs_vars, val_subbasins, seq_len, seq_steps, train_start, train_end, aggregate_daily=agg, include_months=include_month, 
                                                     conv_scalers=train_dataset.scalers, dem=dem, landcover=landcover, soil=soil, groundwater=groundwater, landcover_types=landcover_types)
else:
    val_dataset = train_dataset

# Two test datasets: one with spatial and temporal validation (i.e., different graph, different time), and one with only temporal validation (i.e. different time period only)
spatial_test_dataset = None
if test_subbasins != train_subbasins:
    spatial_test_dataset = datasets.SubbasinAggregatedDataset(rdrs_vars, test_subbasins, seq_len, seq_steps, test_start, test_end, aggregate_daily=agg, include_months=include_month, 
                                                              conv_scalers=train_dataset.scalers, dem=dem, landcover=landcover, soil=soil, groundwater=groundwater, landcover_types=landcover_types)
temporal_test_dataset = datasets.SubbasinAggregatedDataset(rdrs_vars, train_subbasins, seq_len, seq_steps, test_start, test_end, aggregate_daily=agg, include_months=include_month, 
                                                           conv_scalers=train_dataset.scalers, dem=dem, landcover=landcover, soil=soil, groundwater=groundwater, landcover_types=landcover_types)

station_subbasins = train_dataset.simulated_streamflow[~pd.isna(train_dataset.simulated_streamflow['StationID'])]['subbasin'].unique()

Using saved dataset in file /home/mgauch/runoff-nn/src/../data/train_test/SubbasinAggregatedDataset_0-1-2-3-4-5-6-7_14-1_2010-01-15000000-2012-12-31___minmax-minmax-minmax-minmax-sum-minmax-minmax-minmax_month_a173b099769a13df5073c0445e2a7542.pkl
Using saved dataset in file /home/mgauch/runoff-nn/src/../data/train_test/SubbasinAggregatedDataset_0-1-2-3-4-5-6-7_14-1_2010-01-15000000-2012-12-31___minmax-minmax-minmax-minmax-sum-minmax-minmax-minmax_month_638af19f0a2e2c08293bc446025de5a0.pkl
Using saved dataset in file /home/mgauch/runoff-nn/src/../data/train_test/SubbasinAggregatedDataset_0-1-2-3-4-5-6-7_14-1_2013-01-01-2014-12-31___minmax-minmax-minmax-minmax-sum-minmax-minmax-minmax_month_70c2afaf065ec0efccaee01a5e56aa07.pkl
Using saved dataset in file /home/mgauch/runoff-nn/src/../data/train_test/SubbasinAggregatedDataset_0-1-2-3-4-5-6-7_14-1_2013-01-01-2014-12-31___minmax-minmax-minmax-minmax-sum-minmax-minmax-minmax_month_a173b099769a13df5073c0445e2a7542.pkl


In [7]:
# Train model
num_epochs = 1000
learning_rate = 2e-3
patience = 800
min_improvement = 0.01
best_loss_model = (-1, np.inf, None)
dropout = 0.3
weight_decay = 1e-5

batch_size = 16
temp_kernel_size = 3
model = stgcn.Model(train_dataset.x.shape[2], train_adjacency.shape[0], temp_kernel_size, dropout=dropout).to(device)
if num_devices > 1:
    model = torch.nn.DataParallel(model, device_ids=list(range(num_devices)))
loss_fn = evaluate.NSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

writer = SummaryWriter(comment='STGCN_simulationTraining')
param_description = {'time_stamp': time_stamp, 'batch_size': batch_size, 'loss': loss_fn, 'include_month': include_month, 'aggregate_daily': agg, 'rdrs_vars': rdrs_vars, 'dropout': dropout, 'spatial_validation': spatial_val, 'val_fraction': val_fraction, 'temp_kernel': temp_kernel_size,
                     'optimizer': optimizer, 'lr': learning_rate, 'patience': patience, 'min_improvement': min_improvement, 'x_train_shape': train_dataset.x.shape, 'x_val_shape': val_dataset.x.shape, 'partitioning_strategy': partitioning_strategy, 'max_hops': max_hops,
                     'spatial_x_test_shape': spatial_test_dataset.x.shape if spatial_test_dataset is not None else '', 'temporal_x_test_shape': temporal_test_dataset.x.shape, 'num_epochs': num_epochs, 'seq_len': seq_len, 'seq_steps': seq_steps, 'train_start': train_start, 'train_end': train_end, 'weight_decay': weight_decay, 
                     'landcover_types': landcover_types, 'test_start': test_start, 'test_end': test_end, 'model': str(model).replace('\n','').replace(' ', ''),}
writer.add_text('Parameter Description', str(param_description))
str(param_description)

"{'time_stamp': '20190825-153140', 'batch_size': 16, 'loss': NSELoss(), 'include_month': True, 'aggregate_daily': ['minmax', 'minmax', 'minmax', 'minmax', 'sum', 'minmax', 'minmax', 'minmax'], 'rdrs_vars': [0, 1, 2, 3, 4, 5, 6, 7], 'dropout': 0.3, 'spatial_validation': True, 'val_fraction': None, 'temp_kernel': 3, 'optimizer': Adam (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    eps: 1e-08\n    lr: 0.002\n    weight_decay: 1e-05\n), 'lr': 0.002, 'patience': 800, 'min_improvement': 0.01, 'x_train_shape': torch.Size([1082, 14, 82, 328]), 'x_val_shape': torch.Size([1082, 14, 82, 258]), 'partitioning_strategy': 'distance', 'max_hops': 3, 'spatial_x_test_shape': torch.Size([730, 14, 82, 138]), 'temporal_x_test_shape': torch.Size([730, 14, 82, 328]), 'num_epochs': 1000, 'seq_len': 14, 'seq_steps': 1, 'train_start': datetime.datetime(2010, 1, 15, 0, 0), 'train_end': '2012-12-31', 'weight_decay': 1e-05, 'landcover_types': [], 'test_start': '2013-01-01', 'test_end': '2014

In [8]:
if spatial_val:
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True, pin_memory=True, drop_last=False)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size, shuffle=True, pin_memory=True, drop_last=False)
else:
    val_indices = np.random.choice(len(train_dataset), size=int(val_fraction * len(train_dataset)), replace=False)
    train_indices = list(i for i in range(len(train_dataset)) if i not in val_indices)
    train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
    val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, sampler=train_sampler, pin_memory=True, drop_last=False)
    val_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, sampler=val_sampler, pin_memory=True, drop_last=False)
    
if spatial_test_dataset is not None:
    spatial_test_dataloader = torch.utils.data.DataLoader(spatial_test_dataset, batch_size, shuffle=False, pin_memory=True, drop_last=False)
temporal_test_dataloader = torch.utils.data.DataLoader(temporal_test_dataset, batch_size, shuffle=False, pin_memory=True, drop_last=False)

In [9]:
# Subbasins with constant streamflow will divide by zero in loss calculation. Calculate loss without them.
train_non_constant_subbasin_mask = ((train_dataset.y_sim.min(dim=0)[0] - train_dataset.y_sim.max(dim=0)[0]) != 0).to(device)
val_non_constant_subbasin_mask = ((val_dataset.y_sim.min(dim=0)[0] - val_dataset.y_sim.max(dim=0)[0]) != 0).to(device)

y_train_means = train_dataset.y_sim_means.to(device)
y_val_means = val_dataset.y_sim_means[val_non_constant_subbasin_mask].to(device)

In [10]:
torch.manual_seed(0)
np.random.seed(0)
for epoch in range(num_epochs):
    model.train()

    train_losses = torch.tensor(0.0)
    for i, train_batch in enumerate(train_dataloader):
        # in each batch, train on a random subset of connected component to foster generalization
        component_idxs = sorted(n for comp in np.random.choice(train_component_indices, size=len(train_component_indices)//3, replace=False) for n in comp)
        mask = train_non_constant_subbasin_mask[component_idxs]
        adj = train_adjacency[:,component_idxs][:,:,component_idxs]
        y_pred = model(train_batch['x'].permute(0,2,1,3)[...,component_idxs].to(device), adj)
        train_loss = loss_fn(y_pred[:,mask], train_batch['y_sim'][:,component_idxs][:,mask].to(device), means=y_train_means[component_idxs][mask])
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        train_losses += train_loss.detach()
        
    train_loss = (train_losses / len(train_dataloader)).item()
    print('Epoch', epoch, 'mean train loss:\t{}'.format(train_loss))
    writer.add_scalar('loss_nse', train_loss, epoch)
    
    model.eval()
    val_losses = torch.tensor(0.0)
    for i, val_batch in enumerate(val_dataloader):
        y_pred = model(val_batch['x'].permute(0,2,1,3).to(device), val_adjacency).detach()
        val_losses += loss_fn(y_pred[:,val_non_constant_subbasin_mask], val_batch['y_sim'][:,val_non_constant_subbasin_mask].to(device), means=y_val_means).detach()
        
    val_loss = (val_losses / len(val_dataloader)).item()
    print('Epoch', epoch, 'mean val loss:\t{}'.format(val_loss))
    writer.add_scalar('loss_nse_val', val_loss, epoch)
    
    if val_loss < best_loss_model[1] - min_improvement:
        best_loss_model = (epoch, val_loss, model.state_dict())  # new best model
        load_data.pickle_model('STGCN_simulationTraining', model, 'allStations', time_stamp, model_type='torch.dill')
    elif epoch > best_loss_model[0] + patience:
        print('Patience exhausted in epoch {}. Best val-loss was {}'.format(epoch, best_loss_model[1]))
        break
    
print('Using best model from epoch', str(best_loss_model[0]), 'which had loss', str(best_loss_model[1]))
model.load_state_dict(best_loss_model[2])
load_data.save_model_with_state('STGCN_simulationTraining', best_loss_model[0], model, optimizer, time_stamp, use_dill=True)

Epoch 0 mean train loss:	13.929893493652344
Epoch 0 mean val loss:	16.676801681518555
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/STGCN_simulationTraining_allStations_20190825-153140.pkl
Epoch 1 mean train loss:	1.6110553741455078
Epoch 1 mean val loss:	2.7570455074310303
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/STGCN_simulationTraining_allStations_20190825-153140.pkl
Epoch 2 mean train loss:	1.4047967195510864
Epoch 2 mean val loss:	2.5647623538970947
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/STGCN_simulationTraining_allStations_20190825-153140.pkl
Epoch 3 mean train loss:	1.3792766332626343
Epoch 3 mean val loss:	3.135155200958252
Epoch 4 mean train loss:	1.3696426153182983
Epoch 4 mean val loss:	6.130382537841797
Epoch 5 mean train loss:	1.34093177318573
Epoch 5 mean val loss:	5.102980613708496
Epoch 6 mean train loss:	1.2953453063964844
Epoch 6 mean val loss:	2.3612868785858154
Saved model as /home/mgauch/runoff-nn/src/../pickle/mod

In [11]:
del y_train_means, y_val_means, y_pred, train_non_constant_subbasin_mask, val_non_constant_subbasin_mask
if USE_CUDA:
    torch.cuda.empty_cache()

In [12]:
logger.warning('predicting')
model.eval()

temporal_test_predictions = []  # test on train graph but different time
for i, test_batch in enumerate(temporal_test_dataloader):
    pred = model(test_batch['x'].permute(0,2,1,3).to(device), train_adjacency).detach().cpu()
    temporal_test_predictions.append(pred)
predictions = torch.cat(temporal_test_predictions)

if spatial_test_dataset is not None:
    spatial_test_predictions = []  # test on different graph, different time
    for i, test_batch in enumerate(spatial_test_dataloader):
        pred = model(test_batch['x'].permute(0,2,1,3).to(device), test_adjacency).detach().cpu()
        spatial_test_predictions.append(pred)
    predictions = torch.cat([torch.cat(spatial_test_predictions), predictions], dim=1)

2019-08-25 17:02:35,632 - 20190825-153140 - predicting


In [13]:
actuals = temporal_test_dataset.data_streamflow.copy()
if len(actuals['date'].unique()) != len(predictions):
    print('Warning: length of prediction {} and actuals {} does not match.'.format(len(predictions), len(actuals['date'].unique())))

nse_dict, nse_sim_dict = {}, {}
mse_dict, mse_sim_dict = {}, {}
predictions_df = pd.DataFrame(columns=actuals.columns)
predictions_df['is_test_subbasin'] = False
all_test_subbasins = test_subbasins + train_subbasins if train_subbasins != test_subbasins else train_subbasins
for i in range(len(all_test_subbasins)):
    subbasin = all_test_subbasins[i]
    station = None
    subbasin_sim = temporal_test_dataset.simulated_streamflow[temporal_test_dataset.simulated_streamflow['subbasin'] == subbasin].set_index('date')
    if subbasin in station_subbasins:
        station = subbasin_sim['StationID'].values[0]
        act = actuals[actuals['station'] == station].set_index('date')['runoff']
    if predictions.shape[0] != subbasin_sim.shape[0]:
        print('Warning: length of prediction {} and actuals {} does not match for subbasin {}. Ignoring excess actuals.'.format(len(predictions), len(subbasin_sim), subbasin))
        subbasin_sim = subbasin_sim.iloc[:predictions.shape[0]]
        if station is not None:
            act = act.iloc[:predictions.shape[0]]
            
    pred = pd.DataFrame({'runoff': predictions[:,i]}, index=subbasin_sim.index)
    pred['subbasin'] = subbasin
    pred['station'] = station
    pred['is_test_subbasin'] = subbasin in test_subbasins
    predictions_df = predictions_df.append(pred.reset_index(), sort=True)
    subbasin_type = 'test' if subbasin in test_subbasins else 'train'
    nse_sim, mse_sim = evaluate.evaluate_daily('Sub{}'.format(subbasin), pred['runoff'], subbasin_sim['simulated_streamflow'], writer=writer, group=subbasin_type)
    nse_sim_dict[subbasin] = nse_sim
    mse_sim_dict[subbasin] = mse_sim

    if station is not None:
        nse, mse = evaluate.evaluate_daily(station, pred['runoff'], act, writer=writer)
        nse_dict[subbasin] = nse
        mse_dict[subbasin] = mse
        print(station, subbasin, '\tNSE:', nse, '\tMSE:', mse, '(clipped to 0)')
    print(subbasin, '\tNSE sim:', nse_sim, '\tMSE sim:', mse_sim)

/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/pandas/plotting/_converter.py:129: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


1 	NSE sim: -1.1728235785168253 	MSE sim: 2513800.289159647
2 	NSE sim: -1529119.5303730466 	MSE sim: 129787908.05924524
3 	NSE sim: -4087297.386058091 	MSE sim: 130065836.90665291
4 	NSE sim: -5743416.219003963 	MSE sim: 97865045.55232784
5 	NSE sim: -4005407.8437892254 	MSE sim: 97149003.67167918
6 	NSE sim: -21614876.717993006 	MSE sim: 125427758.5215138
7 	NSE sim: -45822113.24890118 	MSE sim: 127047319.11132002
8 	NSE sim: -3283891.4123559785 	MSE sim: 24998605.640492637
9 	NSE sim: -1397079.8575790976 	MSE sim: 5670921.495867866
10 	NSE sim: -981142.0882485353 	MSE sim: 5592353.957787357
11 	NSE sim: -2080855.836507969 	MSE sim: 25392991.719113197
12 	NSE sim: -1971237.9737317069 	MSE sim: 5800952.742675789
13 	NSE sim: -33013940.616231926 	MSE sim: 126701260.04170933
14 	NSE sim: -478497.1137755881 	MSE sim: 128384383.68505672
15 	NSE sim: -49297.66212299163 	MSE sim: 5941801.480728056
16 	NSE sim: -404174.1583298157 	MSE sim: 5913490.086078478
17 	NSE sim: -273595.0949743073 	M

/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/hydroeval/objective_functions.py:31: RuntimeWarning: divide by zero encountered in true_divide
  np.sum((evaluation - np.mean(evaluation)) ** 2, dtype=np.float64))


502 	NSE sim: 0.5413298295390215 	MSE sim: 0.04781753371505817
503 	NSE sim: 0.553250969769473 	MSE sim: 0.012056430971058525
505 	NSE sim: 0.5730128220389179 	MSE sim: 1.2142091433439013
506 	NSE sim: 0.5946320451427964 	MSE sim: 14.254430905155992
507 	NSE sim: 0.5211405627194388 	MSE sim: 0.006669892024084307
508 	NSE sim: 0.5399405013482903 	MSE sim: 1.1989297787604276
509 	NSE sim: 0.5709572436518431 	MSE sim: 24.688669922929556
511 	NSE sim: 0.5389412834174098 	MSE sim: 0.33583578152218324
512 	NSE sim: 0.45386319757078786 	MSE sim: 128.01281721299176
513 	NSE sim: 0.5143465253723432 	MSE sim: 106.6969519145301
514 	NSE sim: 0.6073784250487682 	MSE sim: 0.37237193431526583
515 	NSE sim: 0.5980494342954072 	MSE sim: 0.07889062256689751
516 	NSE sim: 0.5508268066215025 	MSE sim: 0.17305398784576798
517 	NSE sim: 0.5504261805362999 	MSE sim: 0.014439148560281553
518 	NSE sim: 0.5324716578904133 	MSE sim: 0.004263949436433755
519 	NSE sim: 0.46718655359603767 	MSE sim: 1.155280627032

/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/hydroeval/objective_functions.py:31: RuntimeWarning: divide by zero encountered in true_divide
  np.sum((evaluation - np.mean(evaluation)) ** 2, dtype=np.float64))


525 	NSE sim: nan 	MSE sim: 0.004029344771337211
526 	NSE sim: 0.35695566527265354 	MSE sim: 0.0002498056348699059
527 	NSE sim: 0.5806772507353647 	MSE sim: 0.006875483611930098
528 	NSE sim: 0.56106912827 	MSE sim: 0.029646549329092914
529 	NSE sim: 0.5723614070429559 	MSE sim: 0.00440795572855666
531 	NSE sim: -0.4852062016201266 	MSE sim: 1.7752004413570497
532 	NSE sim: -0.06305986515729867 	MSE sim: 0.34941435052560943
533 	NSE sim: 0.3618272466891308 	MSE sim: 0.00125370065865796
534 	NSE sim: 0.5905695498383307 	MSE sim: 0.2503142259726393
538 	NSE sim: -0.3958989195185587 	MSE sim: 238.95594718301007
540 	NSE sim: 0.4345257277801573 	MSE sim: 2948.6218029147167
541 	NSE sim: 0.6125305141688075 	MSE sim: 209.25154248289874
542 	NSE sim: 0.586049662030888 	MSE sim: 236.17833158635034
546 	NSE sim: 0.22013772646448448 	MSE sim: 0.7848286528278592
548 	NSE sim: 0.29204850910539004 	MSE sim: 0.042062338261075125
549 	NSE sim: 0.31932237862713775 	MSE sim: 0.08910335474859483
550 	N

In [14]:
def print_nse_mse(name, nse_dict, mse_dict, subbasins):
    nses = list(nse_dict[s] for s in subbasins)
    mses = list(mse_dict[s] for s in subbasins)
    print(name, 'Median NSE (clipped to 0)', np.nanmedian(nses), '/ Min', np.nanmin(nses), '/ Max', np.nanmax(nses))
    print(' ' * len(name), 'Median MSE (clipped to 0)', np.nanmedian(mses), '/ Min', np.nanmin(mses), '/ Max', np.nanmax(mses))
    
    return np.nanmedian(nses)

nse_median_sim_temporal = print_nse_mse('Temporal test sim', nse_sim_dict, mse_sim_dict, train_subbasins)
nse_median_stations_temporal = print_nse_mse('Stations temporal test', nse_dict, mse_dict, list(s for s in station_subbasins if s in train_subbasins))
if spatial_test_dataset is not None:
    nse_median_sim_spatial = print_nse_mse('Spatial test sim', nse_sim_dict, mse_sim_dict, test_subbasins)
    nse_median_stations_spatial = print_nse_mse('Stations spatial test', nse_dict, mse_dict, list(s for s in station_subbasins if s in test_subbasins))
    writer.add_scalar('nse_median_sim', nse_median_sim_spatial)
    writer.add_scalar('nse_median_stations_spatial', nse_median_stations_spatial)
writer.add_scalar('nse_median_sim_temporal', nse_median_sim_temporal)
writer.add_scalar('nse_median_stations_temporal', nse_median_stations_temporal)

Temporal test sim Median NSE (clipped to 0) 0.4571427978642774 / Min -0.645022691457829 / Max 0.6943772005106627
                  Median MSE (clipped to 0) 6.0857779021546055 / Min 0.0002498056348699059 / Max 55869.44517837075
Stations temporal test Median NSE (clipped to 0) 0.11476348864131836 / Min -0.00944503247916173 / Max 0.2623593933811922
                       Median MSE (clipped to 0) 503.86327933116155 / Min 21.539802144309206 / Max 55084.090236849246
Spatial test sim Median NSE (clipped to 0) -182588.39953616675 / Min -45822113.24890118 / Max 0.27607507934310294
                 Median MSE (clipped to 0) 1879342.7088448554 / Min 0.5604462878124817 / Max 130065836.90665291
Stations spatial test Median NSE (clipped to 0) -78694.02056093508 / Min -3243548.0084769516 / Max -0.14530529010393112
                      Median MSE (clipped to 0) 7527620.435305627 / Min 201.33778659480404 / Max 129744275.1903111


In [15]:
nse_dict

{676: -0.22365184621696432,
 677: -52.52320448899378,
 684: -0.14530529010393112,
 685: -842.430336564334,
 686: -246188.06524862873,
 687: -978.5652591820088,
 688: -61841.34273463738,
 701: -427633.07080557843,
 702: -806702.0657290388,
 703: -3243548.0084769516,
 704: -95546.69838723277,
 705: -46953.26899698353,
 706: -542087.9594115166,
 710: -542664.3204084224,
 707: -0.00944503247916173,
 709: 0.2623593933811922,
 712: 0.20074440932166215,
 713: 0.22198436024359247,
 714: 0.15364144536032343,
 715: 0.10840452630375386,
 716: 0.10468730830330952,
 717: 0.11476348864131836,
 718: 0.10799554194455363,
 723: 0.17950594813111043,
 724: 0.10537655548990865}

In [16]:
writer.close()

In [17]:
save_df = pd.merge(predictions_df.rename({'runoff': 'prediction'}, axis=1), 
                   temporal_test_dataset.simulated_streamflow, on=['date', 'subbasin'])
save_df = pd.merge(save_df, actuals.rename({'runoff': 'actual'}, axis=1), how='left', on=['date', 'station'])\
            [['date', 'subbasin', 'station', 'prediction', 'actual', 'simulated_streamflow', 'is_test_subbasin']]
load_data.pickle_results('STGCN_simulationTraining', save_df, time_stamp)

'STGCN_simulationTraining_20190825-153140.pkl'

In [18]:
_ = print(train_subbasins), print(val_subbasins), print(test_subbasins)

[112, 123, 124, 130, 131, 132, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 151, 152, 153, 154, 155, 156, 158, 159, 160, 161, 162, 163, 164, 165, 166, 169, 170, 171, 172, 173, 174, 175, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 191, 194, 195, 198, 199, 200, 201, 202, 205, 206, 207, 208, 209, 210, 212, 213, 214, 215, 216, 217, 218, 219, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 253, 254, 255, 256, 257, 262, 263, 266, 267, 272, 273, 274, 279, 280, 286, 295, 296, 297, 298, 299, 300, 301, 306, 308, 310, 311, 313, 316, 317, 318, 319, 320, 321, 322, 327, 328, 329, 330, 331, 332, 334, 336, 339, 340, 341, 342, 343, 344, 347, 348, 349, 352, 353, 355, 356, 357, 358, 359, 361, 362, 363, 364, 365, 366, 371, 372, 373, 374, 375, 376, 377, 380, 381, 382, 386, 387, 390, 393, 394, 395, 399, 402, 403, 404, 405, 407, 408, 409, 411, 414, 415, 416, 417, 418, 419, 420, 421,

In [19]:
list(k for k in list(nse_dict.keys()) if k in test_subbasins)

[676, 677, 684, 685, 686, 687, 688, 701, 702, 703, 704, 705, 706, 710]

In [20]:
datetime.now().strftime('%Y%m%d-%H%M%S')

'20190825-170346'